# Aerial Imagery Classifier

Sam Blake, started 23 December 2022

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib notebook
%pylab notebook

import os
import time
import glob
import math
import numpy as np
import cv2
import matplotlib as mpl
from matplotlib import pyplot as plt

from tqdm import tqdm

import numba
from numba import jit, prange, set_num_threads

from PIL import Image

import rasterio
from rasterio.plot import show

from aerial_imagery_classifier import imagery_statistical_binary_classification, \
    show_bgr_image, show_bgra_image, show_greyscale_image 

plt.ioff()
plt.rcParams["figure.figsize"] = (10, 6)

Populating the interactive namespace from numpy and matplotlib


## TavistockAG Precision Weed Trial

In [33]:
dry_crop_1 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_1.jpg')
dry_crop_2 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_2.jpg')
dry_crop_3 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_3.jpg')
dry_crop_4 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_4.jpg')
dry_crop_5 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_5.jpg')
dry_crop_6 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_6.jpg')
dry_crop_7 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_7.jpg')

In [34]:
weed_1 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/weed_1.jpg')

In [35]:
# filename = '../TavistockAG_Precision_Weed_Trial/100_Orthomosaic_rgb_tile_446.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/100_Orthomosaic_rgb_tile_399.tif'
filename = '../TavistockAG_Precision_Weed_Trial/100_Orthomosaic_rgb_tile_658.tif'

In [ ]:
dataset = rasterio.open(filename)

red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
img[:,:,0] = blue
img[:,:,1] = green
img[:,:,2] = red

density, classification_image, classification_binary = \
    imagery_statistical_binary_classification(\
        img, \
        [weed_1], \
        [dry_crop_1, dry_crop_2, dry_crop_3, dry_crop_4, dry_crop_5, dry_crop_6, dry_crop_7], \
        image_resize_ratio = 5, \
        image_blur_kernel_size = (5,5), \
        template_blur_kernel_size = (5,5), \
        template_resize_size = (64,64), \
        duplicate_tolerance = 5, \
        n_sigma_thresholds = [2.,2.25,2.5,2.75,3], \
        equalise = False, \
        denoise_classification_map = True, morph_open = 5, morph_close = 5, \
        export_histogram = True, \
        export_heatmap = True, \
        half_normal_dist = True, \
        imagery_id = filename, \
        plotting = True, \
        verbose = True)

# Export classification map.
cv2.imwrite(filename.replace('.tif',f'_CLASSIFICATION_MAP.PNG'), classification_image)

# Export classification map to GeoTIFF.
output_filename = filename.replace('.tif',f'_CLASSIFICATION_MAP.tif')

kwargs = dataset.meta
kwargs.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw')

with rasterio.open(output_filename, 'w', **kwargs) as dst:
    dst.write_band(1, 255*classification_binary)

In [ ]:
file_pattern = '/Users/user/Documents/Research/rotavision/TavistockAG_Precision_Weed_Trial/tiles/*.tif'
for filename in glob.glob(file_pattern):
    print(os.path.basename(filename))
    dataset = rasterio.open(filename)

    red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
    img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
    img[:,:,0] = blue
    img[:,:,1] = green
    img[:,:,2] = red

    density, classification_image, classification_binary = \
        imagery_statistical_binary_classification(\
            img, \
            [weed_1], \
            [dry_crop_1, dry_crop_2, dry_crop_3, dry_crop_4, dry_crop_5, dry_crop_6, dry_crop_7], \
            image_resize_ratio = 5, \
            image_blur_kernel_size = (5,5), \
            template_blur_kernel_size = (5,5), \
            template_resize_size = (64,64), \
            duplicate_tolerance = 5, \
            n_sigma_thresholds = [2.,2.25,2.5,2.75,3], \
            equalise = False, \
            denoise_classification_map = True, morph_open = 5, morph_close = 5, \
            export_histogram = False, \
            export_heatmap = False, \
            half_normal_dist = True, \
            imagery_id = filename, \
            plotting = False, \
            verbose = False)

    # Export classification map.
    # cv2.imwrite(filename.replace('.tif',f'_CLASSIFICATION_MAP.PNG'), classification_image)

    # Export classification map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_CLASSIFICATION_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, 255*classification_binary)

## nihill radish

Import test imagery:

In [22]:
# filename = 'DJI_20221105165501_0114_V'
# filename = 'DJI_20221105165324_0076_V'
# filename = 'DJI_20221105171447_0579_V'
# filename = 'DJI_20221105165900_0208_V'
# filename = 'DJI_20221105170156_0277_V'
# filename = 'DJI_20221105165643_0154_V'
# filename = 'DJI_20221105171530_0596_V'
# filename = 'DJI_20221105165306_0069_V'
# filename = 'DJI_20221105170235_0292_V'
# filename = 'DJI_20221105171310_0541_V'
# filename = 'DJI_20221105170743_0413_V'
# filename = 'DJI_20221105170442_0342_V'
# filename = 'DJI_20221105170447_0344_V'
# filename = 'DJI_20221105171333_0550_V'
# filename = 'DJI_20221105171507_0587_V'
# filename = 'DJI_20221105171510_0588_V'
# filename = 'DJI_20221105171512_0589_V'
# filename = 'DJI_20221105165033_0009_V'
# filename = 'DJI_20221105165843_0201_V'
# filename = 'DJI_20221105170513_0354_V'
# filename = 'DJI_20221105171436_0575_V'
filename = 'DJI_20221105171312_0542_V'
img2 = cv2.imread(f'test/{filename}.JPG')

In [26]:
image_norm = np.sum(img2, axis=2)

In [27]:
image_norm.shape

(3956, 5280)

Import features for detection: 

In [3]:
nihill_radish_sample_1 = cv2.imread('nihill_radish/nihill_radish_sample_1.jpg')
nihill_radish_sample_2 = cv2.imread('nihill_radish/nihill_radish_sample_2.jpg')
nihill_radish_sample_3 = cv2.imread('nihill_radish/nihill_radish_sample_3.jpg')
nihill_radish_sample_4 = cv2.imread('nihill_radish/nihill_radish_sample_4.jpg')
nihill_radish_sample_5 = cv2.imread('nihill_radish/nihill_radish_sample_5.jpg')
nihill_radish_sample_6 = cv2.imread('nihill_radish/nihill_radish_sample_6.jpg')
nihill_radish_sample_7 = cv2.imread('nihill_radish/nihill_radish_sample_7.jpg')
nihill_radish_sample_1.shape, nihill_radish_sample_2.shape, nihill_radish_sample_3.shape, \
nihill_radish_sample_4.shape, nihill_radish_sample_5.shape, nihill_radish_sample_6.shape, \
nihill_radish_sample_7.shape

((57, 102, 3),
 (69, 71, 3),
 (240, 74, 3),
 (97, 92, 3),
 (183, 106, 3),
 (47, 55, 3),
 (83, 89, 3))

In [4]:
wheat_nov_22_sample_1 = cv2.imread('nihill_wheat_nov_22/nihill_wheat_nov_22_sample_1.jpg')
wheat_nov_22_sample_2 = cv2.imread('nihill_wheat_nov_22/nihill_wheat_nov_22_sample_2.jpg')
wheat_nov_22_sample_3 = cv2.imread('nihill_wheat_nov_22/nihill_wheat_nov_22_sample_3.jpg')
wheat_nov_22_dead_sample_1 = cv2.imread('nihill_wheat_nov_22/nihill_wheat_nov_22_dead_sample_1.jpg')
wheat_nov_22_dead_sample_2 = cv2.imread('nihill_wheat_nov_22/nihill_wheat_nov_22_dead_sample_2.jpg')
wheat_nov_22_sample_1.shape, wheat_nov_22_sample_3.shape, \
wheat_nov_22_dead_sample_1.shape, wheat_nov_22_dead_sample_2.shape

((491, 182, 3), (213, 271, 3), (143, 170, 3), (620, 896, 3))

In [5]:
gravel_road_sample_1 = cv2.imread('gravel_road/gravel_road_sample_1.JPG')
gravel_road_sample_1.shape

(167, 561, 3)

In [ ]:
wheat_nov_22_dead_sample_2

In [ ]:
density, classification_image, spray_region = \
    imagery_statistical_binary_classification(\
        img, \
        [nihill_radish_sample_1, nihill_radish_sample_2, nihill_radish_sample_3, \
        nihill_radish_sample_4, nihill_radish_sample_5, nihill_radish_sample_6, \
        nihill_radish_sample_7], \
        [wheat_nov_22_sample_1, wheat_nov_22_sample_2, wheat_nov_22_sample_3], \
        image_resize_ratio = 5, \
        image_blur_kernel_size = (17,17), \
        template_blur_kernel_size = (17,17), \
        template_resize_size = (32,32), \
        duplicate_tolerance = 5, \
        n_sigma_thresholds = [2.,2.25,2.5,2.75,3], \
        export_histogram = True, \
        export_heatmap = True, \
        half_normal_dist = True, \
        imagery_id = filename, \
        plotting = True, \
        verbose = True)

cv2.imwrite(f'test/{filename}_CLASSIFICATION_MAP_VERSION_5.PNG', classification_image)

In [37]:
weed_map = cv2.imread('test/DJI_20221105170513_0354_V_CLASSIFICATION_MAP_VERSION_4.PNG', cv2.IMREAD_UNCHANGED)

In [ ]:
version_number = 5
plt.ioff()
# file_pattern = '/Users/user/Documents/Research/rotavision/model/test/*V.JPG'
file_pattern = '/Users/user/Documents/Research/rotavision/DJI_202211051646_002_DNihill-Raddish/*.JPG'
for filename in glob.glob(file_pattern):
    if os.path.exists(filename.replace('.JPG',f'_CLASSIFICATION_MAP_VERSION_{version_number}.PNG')):
        continue
    print(os.path.basename(filename))
    img = cv2.imread(filename)
    density, classification_image, spray_region = \
        imagery_statistical_binary_classification(\
            img, \
            [nihill_radish_sample_1, nihill_radish_sample_2, nihill_radish_sample_3, \
            nihill_radish_sample_4, nihill_radish_sample_5, nihill_radish_sample_6, \
            nihill_radish_sample_7], \
            [wheat_nov_22_sample_1, wheat_nov_22_sample_2, wheat_nov_22_sample_3], \
            image_resize_ratio = 5, \
            image_blur_kernel_size = (17,17), \
            template_blur_kernel_size = (17,17), \
            template_resize_size = (32,32), \
            duplicate_tolerance = 5, \
            n_sigma_thresholds = [2.,2.25,2.5,2.75,3], \
            export_histogram = True, \
            export_heatmap = True, \
            half_normal_dist = True, \
            imagery_id = filename, \
            plotting = False, \
            verbose = True)

    # Export classification map.
    cv2.imwrite(filename.replace('.JPG',f'_CLASSIFICATION_MAP_VERSION_{version_number}.PNG'), classification_image)
    
    # Copy EXIF data from original image to classification map.
    image_with_exif = Image.open(filename)
    exif = image_with_exif.info['exif']

    image_wo_exif = Image.open(filename.replace('.JPG',f'_CLASSIFICATION_MAP_VERSION_{version_number}.PNG'))
    image_wo_exif.save(filename.replace('.JPG',f'_CLASSIFICATION_MAP_VERSION_{version_number}.PNG'), 'PNG', exif=exif)

In [ ]:
show_bgra_image(classification_image, axis = 'on')

In [120]:
# Copy EXIF data from original image to classification map.
from PIL import Image
image_with_exif = Image.open('/Users/user/Documents/Research/rotavision/model/test/DJI_20221105165324_0076_V.JPG')
exif = image_with_exif.info['exif']

image_wo_exif = Image.open(\
'/Users/user/Documents/Research/rotavision/model/test/DJI_20221105165324_0076_V_CLASSIFICATION_MAP.PNG')

image_wo_exif.save(\
'/Users/user/Documents/Research/rotavision/model/test/DJI_20221105165324_0076_V_CLASSIFICATION_MAP_TEST.PNG', 
'PNG', exif=exif)